# ***Market Basket Analysis in Python using Apriori Algorithm***

***Implementing Market Basket Analysis***

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori , association_rules


In [ ]:
myData = pd.read_csv('Online Retail.csv', encoding= 'unicode_escape')
myData.head()

In [ ]:
# DATA CLEANING

myData['Description'] = myData['Description'].str.strip()
myData.dropna(axis = 0 , subset = ['InvoiceNo'], inplace = True)
myData['InvoiceNo'] = myData['InvoiceNo'].astype('str')
myData = myData[~myData['InvoiceNo'].str.contains('C')]
myData.head()

In [ ]:
myData['Country'].value_counts()

In [ ]:
myData.shape

In [ ]:
# Seperating Transaction For Germany
myBasket = (myData[myData['Country'] == 'Germany']
            .groupby(['InvoiceNo' , 'Description'])['Quantity']
            .sum().unstack().reset_index().fillna(0)
            .set_index('InvoiceNo')
            )

In [ ]:
myBasket.head()

In [ ]:
# Converting all positive values of 1 and everything else to 0
def my_encodeUnits(x):
  if x <= 0:
    return 0
  elif x >= 1:
    return 1

myBasket_Set = myBasket.applymap(my_encodeUnits)
myBasket_Set.drop('POSTAGE' , inplace = True , axis = 1)

***TRAINING MODEL***

In [ ]:
# Generating Frequent Itemsets
myFrequentItem_Set = apriori(myBasket_Set , min_support = 0.07 , use_colnames= True)

In [ ]:
# Generating Rules
my_Rules = association_rules(myFrequentItem_Set, metric = 'lift' , min_threshold = 1)

In [ ]:
# Viewing top 100 Rules
my_Rules.head(100)

***MAKING RECOMMENDATION***

In [ ]:
myBasket_Set['ROUND SNACK BOXES SET OF4 WOODLAND'].sum()

In [ ]:
myBasket_Set['SPACEBOY LUNCH BOX'].sum()

In [ ]:
# Filtering  Rules Based on Condition
my_Rules[(my_Rules['lift'] >= 3) &
         (my_Rules['confidence'] >= 0.3)]